In [9]:
#install packages
#!pip install --user tqdm

You should consider upgrading via the '/Users/asharifi/anaconda3/bin/python -m pip install --upgrade pip' command.


In [1]:
import kfp
import kfp.components as comp
import kfp.compiler as pipe_comp
import robin_stocks as rs
import pandas as pd
import requests
import numpy as np
import nbimporter
from typing import NamedTuple
from collections import namedtuple
from google.cloud import storage
from datetime import timedelta, datetime
import time
import csv
import dash
from tqdm import tqdm

In [2]:
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.graph_objects as go
import numpy as np

In [3]:
#include functions from notebooks
%run auth.ipynb
%run recent_stock_news.ipynb
%run check_volume.ipynb
%run get_trending_stocks.ipynb
%run purchase_stock.ipynb
%run tradeable_stocks.ipynb

In [4]:
#making notebook functions available
import auth
import recent_stock_news
import check_volume
import get_trending_stocks
import purchase_stock
import tradeable_stocks

Importing Jupyter notebook from auth.ipynb
Importing Jupyter notebook from recent_stock_news.ipynb
Importing Jupyter notebook from check_volume.ipynb
Importing Jupyter notebook from get_trending_stocks.ipynb
Importing Jupyter notebook from purchase_stock.ipynb
Importing Jupyter notebook from tradeable_stocks.ipynb


In [11]:
def push_to_cloud(my_trending_stocks):
    file_name = "purchase_list_V"
    a=my_trending_stocks
    with open(file_name + ".csv", "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerows(a)
    storage_client = storage.Client(project="bright-voltage-248522")
    bucket = storage_client.get_bucket('arys_bucket1')
    blob = bucket.blob("stock_purchase_list")
    blob.upload_from_filename(file_name+".csv")

In [12]:
def get_purchase_list_from_cloud():
    storage_client = storage.Client(project="bright-voltage-248522")
    bucket = storage_client.get_bucket('arys_bucket1')
    blob = bucket.blob("stock_purchase_list")
    data = blob.download_to_filename('stock_purchase_list_d.csv')
    data_csv = pd.read_csv('stock_purchase_list_d.csv', parse_dates=True, names=['symbol', 'news_title', 'trendrate'])
    #sort by trendrate before returning
    data_csv = data_csv.sort_values(by="trendrate", ascending=False)
    
    return data_csv

In [13]:
#runs all the filters to get purchase list
def push_stock_purchase_list_to_cloud():
    all_tradeable_stocks = get_tradeable_stocks()
    print("got all tradeable stocks")
    recent_stock_news1 = filter_recent_news(all_tradeable_stocks)
    print("filterd for recent news")
    low_vol_stocks = remove_high_vol_stocks(recent_stock_news1)
    print("filterd for volume")
    my_trending_stocks = filter_trending_stock(low_vol_stocks)
    print(my_trending_stocks)
    push_to_cloud(my_trending_stocks)
    print("pushed to cloud")

In [14]:
def get_day_trade_stock_list():
    stocks_owned = rs.account.get_current_positions()
    day_trade_list = []
    for stock in stocks_owned:
        stock_url = stock['instrument']
        stock_shares_for_sell = stock['shares_held_for_sells']
        stock_ticker = rs.account.stocks.get_instrument_by_url(url = stock_url)
        if stock_shares_for_sell>1:
            day_trade_list.append(stock_ticker['symbol'])
#could use this to check if a stock has shares held for sells
#get the stock url
    print(day_trade_list)
#use the stock url to get the stock symbol/name
#rs.account.stocks.get_instrument_by_url(url='https://api.robinhood.com/instruments/ee17a597-5d34-42be-b264-f5c50b12dffc/')

In [8]:
get_header = rh_auth(username='', password='')

In [16]:
current_time = datetime.now()
current_time_form = (current_time.hour * 100) + current_time.minute
while current_time_form >= 600 and current_time_form <= 2145:
    #push purchase list to cloud
    print(current_time.hour, current_time.minute)

    push_stock_purchase_list_to_cloud()
    purchase_list = get_purchase_list_from_cloud()
    if len(purchase_list) > 0:
        buy_stock(purchase_list)

    current_time = datetime.now()
    print(current_time.hour, current_time.minute)
    current_time_form = (current_time.hour * 100) + current_time.minute

19 17
Found Additional pages.
Loading page 2 ...
Loading page 3 ...
Loading page 4 ...
Loading page 5 ...
Loading page 6 ...
Loading page 7 ...
Loading page 8 ...
Loading page 9 ...
Loading page 10 ...
Loading page 11 ...
Loading page 12 ...
Loading page 13 ...
Loading page 14 ...
Loading page 15 ...
Loading page 16 ...
Loading page 17 ...
Loading page 18 ...
Loading page 19 ...
Loading page 20 ...
Loading page 21 ...
Loading page 22 ...
Loading page 23 ...
Loading page 24 ...
Loading page 25 ...
Loading page 26 ...
Loading page 27 ...
Loading page 28 ...
Loading page 29 ...
Loading page 30 ...
Loading page 31 ...
Loading page 32 ...
Loading page 33 ...
Loading page 34 ...
Loading page 35 ...
Loading page 36 ...
Loading page 37 ...
Loading page 38 ...
Loading page 39 ...
Loading page 40 ...
Loading page 41 ...
Additional pages exist but could not be loaded.
Found 4100 results
got all tradeable stocks


KeyboardInterrupt: 